In [1]:
import numpy as np
import time

In [2]:
filename_i = "bram_input_init_"
def append_hex(a, b,size):
    sizeof_b = 8*size
    return (a << sizeof_b) | b

def bram_input_init(input, width, height, channel):
    quarterNumberOfEntries = int(width*height*channel/4)
    for i in range(4):
        filename_temp = filename_i + str(i) + ".coe"
        f = open(filename_temp, "w")
        f.write("memory_initialization_radix=16;\nmemory_initialization_vector=")
        count = 0
        temp = 0x0
        for j in range(quarterNumberOfEntries):
            temp = append_hex(input[i*quarterNumberOfEntries+j],temp,count)
            count = count + 1
            if(count == 4):
                f.write(str(hex(temp)[2:])+ " ")
                temp = 0x0
                count = 0
    if (quarterNumberOfEntries % 4 != 0):
        f.write(str(hex(temp)[2:])+" ")
    f.write(";")
    f.close()



In [26]:
filename_w = "bram_weight_init_"

def append_hex(a, b,size):
    sizeof_b = 8*size
    return (a << sizeof_b) | b
    
def bram_weight_init(weight, kernel, width, height, channel):
	numberOfEntries = width * height

	quarterOfChannel = channel/4
	quarterOfKernel = kernel/4
	entriesInOneFilter = width * height
	entriesInOneKernel = width * height * channel

	starting_point = 0
	file_count = 0
	file_index = [0,4,8,12,1,5,9,13,2,6,10,14,3,7,11,15]
	for i in file_index:
         filename_temp = filename_w + str(i) + ".coe"
         f = open(filename_temp, "w")
         f.write("memory_initialization_radix=16;\nmemory_initialization_vector=")
         count = 0
         temp = 0x0
         for j in range (int(quarterOfKernel)):
             for m in range(int(quarterOfChannel)):
                 for n in range(entriesInOneFilter):
                     temp = append_hex(weight[int(starting_point + n + m*entriesInOneFilter + j*entriesInOneKernel)]%256,temp,count)
                     count = count + 1
                     if(count == 4):
                         f.write(str(hex(temp)[2:])+ " ")
                         temp = 0x0
                         count = 0
         if (int(numberOfEntries*quarterOfKernel*quarterOfChannel) % 4 != 0):
             f.write(str(hex(temp)[2:])+" ")
         f.write(";")
         f.close()
         file_count = file_count + 1
         if (file_count%4 == 0):
             starting_point = starting_point + entriesInOneFilter*quarterOfChannel + entriesInOneKernel*(quarterOfKernel-1)
         else:
             starting_point = starting_point + entriesInOneFilter*quarterOfChannel


In [4]:
def reshape_data(data, dtypes, m, n_C, n_W, n_H):
    n_data = np.array(data, dtype = dtypes)
    n_data = np.reshape(n_data, (m,n_C,n_W,n_H))
    return n_data

In [5]:
def conv(m, n_C, n_W_prev, n_H_prev, f,  stride, n_feature, n_weight): #parameters: weight, kernel, width, height, channel
    pad = 0
    t = []
    n_H = int((n_H_prev + 2*pad - f)/stride) + 1
    n_W = int((n_W_prev + 2*pad - f)/stride) + 1
    time3 = time.time()
    for i in range(m):
        for c in range(n_C):

            for h in range (n_H):
                vert_start = stride * h 
                vert_end = vert_start  + f
      
                for w in range (n_W):
                    horiz_start = stride * w
                    horiz_end = horiz_start + f

                    temp_feature = np.array(n_feature[0, 0,vert_start:vert_end, horiz_start:horiz_end ], dtype = np.uint8)
                    #print("F: ", temp_feature)
                    w = np.array(n_weight[0, c, :, :], dtype = np.uint8)
                    #print("W: ", n_weight[0, c, :, :])
                    temp_ans = np.multiply(temp_feature, w, dtype = np.uint8)
                    t.append(np.sum(temp_ans, dtype = np.uint8))
                    #print(t)
    time4 = time.time()
    print("Calculation time: ", time4 - time3)
    return t;

In [16]:
#input specs
stride = 1
pad = 0
m = 1 #number of pictures
n_C_prev = 8 #number of channels of picture
n_C = 8 #number of kernel
n_H_prev, n_W_prev = 5, 5 #size of picture
f = 3 #size of kernel

weight: 8 kernel, 8 channel, 4x4

In [27]:
feature = list(range(26,226))
weight = list(range(1,577))
bram_input_init(feature,n_H_prev,n_W_prev,n_C_prev) # parameters: input array, width, height, channel
bram_weight_init(weight,n_C,3,3,n_C_prev) #parameters: weight, kernel%4, width, height, channel%4
n_f = reshape_data(feature, np.uint8,m ,n_C_prev, n_H_prev, n_W_prev)
n_w = reshape_data(weight, np.uint8, n_C, n_C_prev, f, f)
a = conv(m, n_C, n_H_prev, n_W_prev, f, 1, n_f, n_w)

Calculation time:  0.0010001659393310547


In [28]:
a

[0,
 45,
 90,
 225,
 14,
 59,
 194,
 239,
 28,
 32,
 158,
 28,
 150,
 20,
 146,
 12,
 138,
 8,
 64,
 15,
 222,
 75,
 26,
 233,
 86,
 37,
 244,
 96,
 128,
 160,
 0,
 32,
 64,
 160,
 192,
 224,
 128,
 241,
 98,
 181,
 38,
 151,
 234,
 91,
 204,
 160,
 98,
 36,
 106,
 44,
 238,
 52,
 246,
 184,
 192,
 211,
 230,
 31,
 50,
 69,
 126,
 145,
 164,
 224,
 68,
 168,
 212,
 56,
 156,
 200,
 44,
 144]